# **Chapter 18: Web Application Testing Types**

---

## **18.1 Functional Testing**

### **What is Functional Testing?**

**Functional Testing** verifies that each function of the web application operates in conformance with the requirement specification. It focuses on **what the system does** rather than how it does it (which is non-functional testing).

**Scope:** Testing business logic, user workflows, data manipulation, calculations, and feature completeness.

**Analogy:** Like testing a car's functions—does the steering wheel turn the car? Do the brakes stop it? Does the horn honk? Functional testing checks each feature works as designed.

### **Functional Testing Strategy**

```
Functional Testing Pyramid:

                    ┌─────────────┐
                    │   E2E Tests │  ← User journeys (few)
                    │  (Critical) │
                    └──────┬──────┘
                           │
                    ┌──────▼──────┐
                    │  Integration │  ← Feature workflows
                    │    Tests     │
                    └──────┬──────┘
                           │
                    ┌──────▼──────┐
                    │   Unit Tests │  ← Business logic (many)
                    │  (Components)│
                    └─────────────┘
```

### **Implementation Examples**

#### **User Workflow Testing (E2E)**

```python
# Selenium - Complete purchase workflow
class TestPurchaseWorkflow:
    """End-to-end functional test of e-commerce purchase"""
    
    def test_complete_purchase(self, driver):
        # Step 1: Browse products
        driver.get("https://shop.example.com")
        search = driver.find_element(By.ID, "search")
        search.send_keys("laptop")
        search.submit()
        
        # Verify search results
        results = driver.find_elements(By.CLASS_NAME, "product-card")
        assert len(results) > 0, "No products found"
        
        # Step 2: Select product
        results[0].click()
        assert "laptop" in driver.title.lower()
        
        # Step 3: Add to cart
        driver.find_element(By.ID, "add-to-cart").click()
        
        # Verify cart update
        cart_count = driver.find_element(By.ID, "cart-count").text
        assert cart_count == "1"
        
        # Step 4: Checkout
        driver.find_element(By.ID, "checkout").click()
        
        # Fill shipping
        driver.find_element(By.ID, "name").send_keys("Test User")
        driver.find_element(By.ID, "address").send_keys("123 Test St")
        driver.find_element(By.ID, "city").send_keys("Test City")
        
        # Select country
        Select(driver.find_element(By.ID, "country")).select_by_value("US")
        
        # Step 5: Payment
        driver.find_element(By.ID, "card-number").send_keys("4111111111111111")
        driver.find_element(By.ID, "expiry").send_keys("12/25")
        driver.find_element(By.ID, "cvv").send_keys("123")
        
        # Submit
        driver.find_element(By.ID, "place-order").click()
        
        # Step 6: Verification
        wait = WebDriverWait(driver, 10)
        confirmation = wait.until(
            EC.presence_of_element_located((By.ID, "order-confirmation"))
        )
        
        assert "Thank you" in confirmation.text
        order_number = driver.find_element(By.ID, "order-number").text
        assert len(order_number) > 0
        
        # Verify email sent (check via API or email service)
        email = get_last_email("test@example.com")
        assert order_number in email.subject
```

#### **API-Level Functional Testing**

```python
# Integration testing of business logic via API
import requests
import pytest

class TestUserManagementAPI:
    """Functional tests for user management backend"""
    
    BASE_URL = "https://api.example.com/v1"
    
    def test_create_user_workflow(self):
        """Complete user lifecycle: create → verify → update → delete"""
        
        # Step 1: Create user
        create_payload = {
            "username": f"testuser_{uuid.uuid4().hex[:8]}",
            "email": "test@example.com",
            "role": "standard"
        }
        
        create_resp = requests.post(
            f"{self.BASE_URL}/users",
            json=create_payload
        )
        assert create_resp.status_code == 201
        user_id = create_resp.json()["id"]
        
        # Step 2: Verify user created
        get_resp = requests.get(f"{self.BASE_URL}/users/{user_id}")
        assert get_resp.status_code == 200
        user_data = get_resp.json()
        assert user_data["username"] == create_payload["username"]
        assert user_data["status"] == "active"
        
        # Step 3: Update user
        update_resp = requests.patch(
            f"{self.BASE_URL}/users/{user_id}",
            json={"role": "admin"}
        )
        assert update_resp.status_code == 200
        assert update_resp.json()["role"] == "admin"
        
        # Step 4: Verify permissions changed
        perms_resp = requests.get(
            f"{self.BASE_URL}/users/{user_id}/permissions",
            headers={"Authorization": f"Bearer {admin_token}"}
        )
        assert "admin_access" in perms_resp.json()["permissions"]
        
        # Step 5: Delete user
        delete_resp = requests.delete(f"{self.BASE_URL}/users/{user_id}")
        assert delete_resp.status_code == 204
        
        # Step 6: Verify deletion
        verify_resp = requests.get(f"{self.BASE_URL}/users/{user_id}")
        assert verify_resp.status_code == 404
```

---

## **18.2 UI/UX Testing**

### **What is UI/UX Testing?**

**UI (User Interface) Testing** verifies that visual elements (buttons, text, images, layout) appear correctly and function as designed.

**UX (User Experience) Testing** evaluates the overall experience—ease of use, intuitiveness, accessibility, and user satisfaction.

**Analogy:** UI testing checks that a car's dashboard has all the gauges in the right places with correct labels. UX testing checks whether the driver can easily read the gauges while driving and understand what they mean without confusion.

### **UI Testing Scope**

```python
# Visual validation checklist
class TestUIValidation:
    """Comprehensive UI testing"""
    
    def test_element_visibility(self, driver):
        """Elements visible and properly rendered"""
        driver.get("https://example.com")
        
        # Logo visible
        logo = driver.find_element(By.ID, "logo")
        assert logo.is_displayed()
        assert logo.size["height"] > 0  # Has dimensions
        
        # Navigation menu visible
        nav = driver.find_element(By.TAG_NAME, "nav")
        assert nav.is_displayed()
        
        # Check specific elements exist
        assert driver.find_element(By.ID, "search-box")
        assert driver.find_element(By.ID, "user-menu")
    
    def test_text_content(self, driver):
        """Text displays correctly"""
        driver.get("https://example.com")
        
        # Heading text
        heading = driver.find_element(By.TAG_NAME, "h1")
        assert heading.text == "Welcome to Example"
        
        # No broken text (encoding issues)
        special_chars = driver.find_element(By.ID, "special")
        assert "café" in special_chars.text  # UTF-8 handling
        assert "日本語" in special_chars.text  # Unicode
        
        # Dynamic text loaded via AJAX
        wait = WebDriverWait(driver, 10)
        dynamic = wait.until(EC.presence_of_element_located((By.ID, "dynamic-content")))
        assert len(dynamic.text) > 0
    
    def test_image_rendering(self, driver):
        """Images load correctly"""
        driver.get("https://example.com")
        
        images = driver.find_elements(By.TAG_NAME, "img")
        
        for img in images:
            # Image loaded (naturalWidth > 0)
            is_loaded = driver.execute_script(
                "return arguments[0].complete && typeof arguments[0].naturalWidth != 'undefined' && arguments[0].naturalWidth > 0",
                img
            )
            assert is_loaded, f"Image failed to load: {img.get_attribute('src')}"
            
            # Alt text present (accessibility)
            assert img.get_attribute("alt"), "Missing alt text"
    
    def test_responsive_layout(self, driver):
        """Layout adapts to viewport"""
        # Desktop
        driver.set_window_size(1920, 1080)
        driver.get("https://example.com")
        sidebar = driver.find_element(By.ID, "sidebar")
        assert sidebar.is_displayed()
        
        # Tablet
        driver.set_window_size(768, 1024)
        driver.refresh()
        # Sidebar might be collapsed or repositioned
        menu_btn = driver.find_element(By.ID, "menu-toggle")
        assert menu_btn.is_displayed()
        
        # Mobile
        driver.set_window_size(375, 667)
        driver.refresh()
        # Hamburger menu should be present
        hamburger = driver.find_element(By.CSS_SELECTOR, ".hamburger-menu")
        assert hamburger.is_displayed()
        # Sidebar hidden
        sidebar = driver.find_element(By.ID, "sidebar")
        assert not sidebar.is_displayed() or sidebar.size["width"] == 0
    
    def test_color_and_styling(self, driver):
        """Visual appearance correct"""
        driver.get("https://example.com")
        
        button = driver.find_element(By.ID, "primary-btn")
        
        # Background color
        bg_color = button.value_of_css_property("background-color")
        # Convert rgb to hex if needed
        assert "rgb(0, 123, 255)" in bg_color or "#007bff" in bg_color
        
        # Font properties
        font_size = button.value_of_css_property("font-size")
        assert font_size == "16px"
        
        # Border radius
        border_radius = button.value_of_css_property("border-radius")
        assert border_radius == "4px"
        
        # Hover state (requires ActionChains)
        from selenium.webdriver.common.action_chains import ActionChains
        actions = ActionChains(driver)
        actions.move_to_element(button).perform()
        
        # Check hover color change
        hover_color = button.value_of_css_property("background-color")
        assert hover_color != bg_color  # Should change on hover
```

### **UX Testing Techniques**

```python
class TestUX:
    """User Experience testing"""
    
    def test_task_completion_time(self, driver):
        """Measure time to complete key tasks"""
        import time
        
        driver.get("https://example.com")
        
        start = time.time()
        
        # Complete checkout flow
        driver.find_element(By.ID, "add-to-cart").click()
        driver.find_element(By.ID, "checkout").click()
        driver.find_element(By.ID, "name").send_keys("Test User")
        driver.find_element(By.ID, "card").send_keys("4111111111111111")
        driver.find_element(By.ID, "place-order").click()
        
        # Wait for confirmation
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "confirmation"))
        )
        
        end = time.time()
        duration = end - start
        
        # UX standard: Checkout should complete in < 30 seconds
        assert duration < 30, f"Checkout took {duration}s, target is <30s"
    
    def test_error_recovery(self, driver):
        """Can users recover from errors?"""
        driver.get("https://example.com/form")
        
        # Submit empty form (error)
        driver.find_element(By.ID, "submit").click()
        
        # Error message visible?
        error = driver.find_element(By.CLASS_NAME, "error-message")
        assert error.is_displayed()
        assert "required" in error.text.lower()
        
        # Can user correct and resubmit?
        driver.find_element(By.ID, "email").send_keys("test@example.com")
        driver.find_element(By.ID, "submit").click()
        
        # Success this time?
        success = driver.find_element(By.CLASS_NAME, "success-message")
        assert success.is_displayed()
    
    def test_navigation_consistency(self, driver):
        """Navigation is intuitive and consistent"""
        driver.get("https://example.com")
        
        # Main navigation present on all pages?
        pages = ["/", "/about", "/products", "/contact"]
        
        for page in pages:
            driver.get(f"https://example.com{page}")
            
            # Logo links home
            logo = driver.find_element(By.ID, "logo")
            assert logo.get_attribute("href") == "https://example.com/"
            
            # Navigation menu present
            nav = driver.find_element(By.TAG_NAME, "nav")
            assert nav.is_displayed()
            
            # Current page highlighted in nav
            current_nav = driver.find_element(By.CSS_SELECTOR, f"nav a[href='{page}']")
            assert "active" in current_nav.get_attribute("class")
    
    def test_accessibility_basic(self, driver):
        """Basic accessibility checks"""
        driver.get("https://example.com")
        
        # Images have alt text
        images = driver.find_elements(By.TAG_NAME, "img")
        for img in images:
            alt = img.get_attribute("alt")
            assert alt and len(alt) > 0, f"Image missing alt: {img.get_attribute('src')}"
        
        # Form labels present
        inputs = driver.find_elements(By.CSS_SELECTOR, "input:not([type='hidden'])")
        for input_field in inputs:
            input_id = input_field.get_attribute("id")
            # Check for associated label
            labels = driver.find_elements(By.CSS_SELECTOR, f"label[for='{input_id}']")
            aria_label = input_field.get_attribute("aria-label")
            aria_labelled_by = input_field.get_attribute("aria-labelledby")
            
            assert len(labels) > 0 or aria_label or aria_labelled_by, \
                f"Input {input_id} missing label"
        
        # Keyboard navigation (Tab order)
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.TAB)
        # Check focus moved to expected element
        focused = driver.switch_to.active_element
        assert focused.get_attribute("id") == "first-focusable-element"
        
        # Color contrast (basic check)
        # Note: Use axe-core or similar for proper contrast testing
        heading = driver.find_element(By.TAG_NAME, "h1")
        color = heading.value_of_css_property("color")
        bg_color = heading.value_of_css_property("background-color")
        # Should check contrast ratio >= 4.5:1 for normal text
```

---

## **18.2 Cross-Browser Testing**

### **Strategy and Implementation**

```python
# Cross-browser testing with Selenium using pytest parametrization
import pytest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.edge.options import Options as EdgeOptions

@pytest.fixture(params=["chrome", "firefox", "edge"])
def cross_browser_driver(request):
    browser = request.param
    
    if browser == "chrome":
        options = ChromeOptions()
        options.add_argument("--start-maximized")
        driver = webdriver.Chrome(options=options)
    elif browser == "firefox":
        options = FirefoxOptions()
        driver = webdriver.Firefox(options=options)
    elif browser == "edge":
        options = EdgeOptions()
        driver = webdriver.Edge(options=options)
    
    yield driver
    driver.quit()

def test_login_works_all_browsers(cross_browser_driver):
    """Verify login works consistently across browsers"""
    driver = cross_browser_driver
    driver.get("https://example.com/login")
    
    # These operations should work identically across browsers
    driver.find_element(By.ID, "username").send_keys("test")
    driver.find_element(By.ID, "password").send_keys("pass")
    driver.find_element(By.ID, "submit").click()
    
    assert "dashboard" in driver.current_url
```

### **Cross-Browser Compatibility Issues**

```python
class TestCrossBrowserCompatibility:
    """Common cross-browser issues and solutions"""
    
    def test_css_rendering_consistency(self, driver):
        """Verify CSS renders consistently"""
        driver.get("https://example.com")
        
        element = driver.find_element(By.ID, "styled-box")
        
        # Check computed styles (may vary slightly by browser)
        bg_color = element.value_of_css_property("background-color")
        # Chrome: "rgba(0, 123, 255, 1)"
        # Firefox: "rgb(0, 123, 255)"
        
        # Normalize for comparison
        assert "0, 123, 255" in bg_color or "007bff" in bg_color.lower()
    
    def test_javascript_execution(self, driver):
        """JavaScript may behave differently"""
        driver.get("https://example.com")
        
        # Date parsing differences
        result = driver.execute_script("""
            return new Date('2023-01-01').toISOString();
        """)
        
        # Should be consistent but verify format
        assert result.startswith("2023-01-01")
    
    def test_form_submission(self, driver):
        """Form behavior varies by browser"""
        driver.get("https://example.com/form")
        
        # Some browsers auto-validate HTML5 inputs
        email_field = driver.find_element(By.ID, "email")
        email_field.send_keys("invalid-email")
        
        submit = driver.find_element(By.ID, "submit")
        submit.click()
        
        # Check if browser prevented submission (HTML5 validation)
        # or if server returned error
        if "invalid" in driver.current_url:
            # Server-side validation caught it
            pass
        else:
            # Check for HTML5 validation message
            validation_msg = driver.execute_script(
                "return arguments[0].validationMessage;", email_field
            )
            assert "@" in validation_msg or "email" in validation_msg.lower()
    
    def test_file_download(self, driver):
        """Download behavior varies significantly"""
        # Chrome setup
        chrome_options = webdriver.ChromeOptions()
        prefs = {
            "download.default_directory": "/tmp/downloads",
            "download.prompt_for_download": False
        }
        chrome_options.add_experimental_option("prefs", prefs)
        
        # Firefox setup (completely different)
        firefox_options = webdriver.FirefoxOptions()
        firefox_options.set_preference("browser.download.folderList", 2)
        firefox_options.set_preference("browser.download.dir", "/tmp/downloads")
        firefox_options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")
        
        # Test must handle both differently
```

---

## **18.3 Responsive Design Testing**

### **Viewport Testing**

```python
class TestResponsiveDesign:
    """Testing responsive layouts"""
    
    # Standard breakpoints
    BREAKPOINTS = {
        "mobile_small": (320, 568),    # iPhone SE
        "mobile": (375, 667),          # iPhone 8
        "mobile_large": (414, 896),    # iPhone 11 Pro Max
        "tablet": (768, 1024),         # iPad
        "tablet_large": (1024, 1366),  # iPad Pro
        "desktop": (1920, 1080),       # Full HD
        "desktop_large": (2560, 1440)  # 2K
    }
    
    def test_layout_adapts_to_viewport(self, driver):
        """Verify layout changes appropriately"""
        for device, (width, height) in self.BREAKPOINTS.items():
            driver.set_window_size(width, height)
            driver.get("https://responsive-site.example.com")
            
            if width < 768:
                # Mobile: Hamburger menu should be present
                menu = driver.find_element(By.CSS_SELECTOR, ".hamburger-menu")
                assert menu.is_displayed()
                
                # Sidebar should be hidden or collapsed
                try:
                    sidebar = driver.find_element(By.ID, "sidebar")
                    assert not sidebar.is_displayed() or sidebar.size["width"] < 100
                except NoSuchElementException:
                    pass  # Sidebar removed from DOM on mobile
                
            elif width < 1024:
                # Tablet: Adjusted layout
                # Maybe 2-column grid instead of 3
                grid_items = driver.find_elements(By.CLASS_NAME, "grid-item")
                # Check computed styles for width percentage
                
            else:
                # Desktop: Full layout
                sidebar = driver.find_element(By.ID, "sidebar")
                assert sidebar.is_displayed()
                assert sidebar.size["width"] > 200
    
    def test_images_responsive(self, driver):
        """Verify responsive images load appropriate sizes"""
        driver.set_window_size(375, 667)  # Mobile
        driver.get("https://example.com")
        
        # Check srcset or picture element
        img = driver.find_element(By.ID, "hero-image")
        
        # Get current src
        src = img.get_attribute("src")
        
        # Should load mobile-optimized version
        assert "mobile" in src or "375" in src or "small" in src
        
        # Resize to desktop
        driver.set_window_size(1920, 1080)
        driver.refresh()
        
        img = driver.find_element(By.ID, "hero-image")
        src = img.get_attribute("src")
        
        # Should load high-res version
        assert "desktop" in src or "1920" in src or "large" in src
    
    def test_touch_events_mobile(self, driver):
        """Verify touch interactions on mobile"""
        from selenium.webdriver.common.touch_actions import TouchActions
        
        driver.set_window_size(375, 667)
        
        # Enable touch emulation (Chrome)
        driver.execute_cdp_cmd('Emulation.setTouchEmulationEnabled', {
            'enabled': True
        })
        
        # Swipe gesture
        element = driver.find_element(By.ID, "swipe-area")
        
        touch = TouchActions(driver)
        touch.tap_and_hold(element)
        touch.move(100, 0)  # Move right
        touch.release(element)
        touch.perform()
        
        # Verify swipe triggered action
        assert driver.find_element(By.ID, "swiped-content").is_displayed()
    
    def test_font_readability(self, driver):
        """Verify text is readable at all sizes"""
        for width, height in [(375, 667), (768, 1024), (1920, 1080)]:
            driver.set_window_size(width, height)
            driver.get("https://example.com")
            
            # Check font sizes are readable (>= 12px)
            paragraphs = driver.find_elements(By.TAG_NAME, "p")
            for p in paragraphs:
                font_size = p.value_of_css_property("font-size")
                size_px = int(font_size.replace("px", ""))
                assert size_px >= 12, f"Font too small: {size_px}px"
            
            # Check contrast (basic check)
            # Use axe-core or similar for proper contrast testing
```

---

## **18.4 Cross-Browser Testing**

### **Comprehensive Cross-Browser Strategy**

```python
class TestCrossBrowser:
    """Systematic cross-browser validation"""
    
    BROWSERS = {
        "chrome": {
            "driver": webdriver.Chrome,
            "options": ChromeOptions,
            "features": ["headless", "extensions", "devtools"]
        },
        "firefox": {
            "driver": webdriver.Firefox,
            "options": FirefoxOptions,
            "features": ["headless", "profile"]
        },
        "safari": {
            "driver": webdriver.Safari,
            "options": None,  # Limited options
            "features": ["automation"]
        },
        "edge": {
            "driver": webdriver.Edge,
            "options": EdgeOptions,
            "features": ["headless", "ie-mode"]
        }
    }
    
    def test_rendering_consistency(self, driver_factory):
        """Verify visual consistency across browsers"""
        test_cases = [
            {
                "name": "Login Form",
                "url": "/login",
                "checks": ["form_layout", "input_styling", "button_appearance"]
            },
            {
                "name": "Data Table",
                "url": "/users",
                "checks": ["table_layout", "pagination", "sorting_icons"]
            }
        ]
        
        results = {}
        
        for browser_name, config in self.BROWSERS.items():
            results[browser_name] = {}
            
            for case in test_cases:
                driver = driver_factory(browser_name)
                try:
                    driver.get(f"https://example.com{case['url']}")
                    
                    # Take screenshot for comparison
                    screenshot_path = f"screenshots/{browser_name}_{case['name']}.png"
                    driver.save_screenshot(screenshot_path)
                    
                    # Run checks
                    for check in case['checks']:
                        result = self.run_visual_check(driver, check)
                        results[browser_name][f"{case['name']}_{check}"] = result
                        
                finally:
                    driver.quit()
        
        # Compare results across browsers
        self.assert_cross_browser_consistency(results)
    
    def test_javascript_execution(self, driver_factory):
        """JavaScript behaves differently across browsers"""
        browsers = ["chrome", "firefox", "safari"]
        
        for browser in browsers:
            driver = driver_factory(browser)
            
            # Date parsing (known differences)
            result = driver.execute_script("""
                return new Date('2023-01-01T00:00:00').toISOString();
            """)
            
            # Should be consistent but document any differences
            print(f"{browser}: {result}")
            
            # CSS property access (camelCase vs hyphenated)
            element = driver.find_element(By.ID, "test")
            bg_color = element.value_of_css_property("background-color")
            
            driver.quit()
    
    def test_form_submission_behavior(self, driver_factory):
        """Form validation and submission varies"""
        for browser in ["chrome", "firefox", "safari"]:
            driver = driver_factory(browser)
            driver.get("https://example.com/form")
            
            # HTML5 validation behavior
            email = driver.find_element(By.ID, "email")
            email.send_keys("invalid-email")
            
            submit = driver.find_element(By.ID, "submit")
            submit.click()
            
            # Check if browser blocked submission (HTML5 validation)
            # or if server returned error
            current_url = driver.current_url
            validation_message = driver.execute_script(
                "return arguments[0].validationMessage;", email
            )
            
            print(f"{browser}: URL={current_url}, Validation='{validation_message}'")
            
            driver.quit()
```

---

## **18.5 Performance Testing for Web**

### **Web Performance Metrics**

```python
class TestWebPerformance:
    """Client-side performance testing"""
    
    def test_page_load_metrics(self, driver):
        """Measure Core Web Vitals and load times"""
        driver.get("https://example.com")
        
        # Execute performance timing API
        timing = driver.execute_script("""
            return window.performance.timing;
        """)
        
        navigation_start = timing['navigationStart']
        load_event_end = timing['loadEventEnd']
        dom_complete = timing['domComplete']
        
        # Calculate metrics
        page_load_time = load_event_end - navigation_start
        dom_ready_time = dom_complete - navigation_start
        
        print(f"Page Load: {page_load_time}ms")
        print(f"DOM Ready: {dom_ready_time}ms")
        
        # Assert performance budgets
        assert page_load_time < 3000, f"Page load too slow: {page_load_time}ms"
        assert dom_ready_time < 1500, f"DOM ready too slow: {dom_ready_time}ms"
    
    def test_core_web_vitals(self, driver):
        """Measure LCP, FID, CLS"""
        driver.get("https://example.com")
        
        # Largest Contentful Paint (LCP)
        lcp = driver.execute_script("""
            return new Promise((resolve) => {
                new PerformanceObserver((list) => {
                    const entries = list.getEntries();
                    const lastEntry = entries[entries.length - 1];
                    resolve(lastEntry.startTime);
                }).observe({ entryTypes: ['largest-contentful-paint'] });
            });
        """)
        
        print(f"LCP: {lcp}ms")
        assert lcp < 2500, f"LCP too slow: {lcp}ms"
        
        # Cumulative Layout Shift (CLS)
        cls = driver.execute_script("""
            return new Promise((resolve) => {
                let cls = 0;
                new PerformanceObserver((list) => {
                    for (const entry of list.getEntries()) {
                        if (!entry.hadRecentInput) {
                            cls += entry.value;
                        }
                    }
                }).observe({ entryTypes: ['layout-shift'] });
                
                setTimeout(() => resolve(cls), 5000);
            });
        """)
        
        print(f"CLS: {cls}")
        assert cls < 0.1, f"CLS too high: {cls}"
    
    def test_resource_loading(self, driver):
        """Check for render-blocking resources"""
        driver.get("https://example.com")
        
        resources = driver.execute_script("""
            return performance.getEntriesByType('resource').map(r => ({
                name: r.name,
                type: r.initiatorType,
                duration: r.duration,
                size: r.transferSize
            }));
        """)
        
        # Check for large images
        images = [r for r in resources if r['type'] == 'img']
        for img in images:
            if img['size'] > 500000:  # 500KB
                print(f"WARNING: Large image {img['name']}: {img['size']} bytes")
        
        # Check for slow resources
        slow = [r for r in resources if r['duration'] > 1000]
        for resource in slow:
            print(f"SLOW: {resource['name']} took {resource['duration']}ms")
    
    def test_time_to_interactive(self, driver):
        """Measure when page becomes fully interactive"""
        driver.get("https://example.com")
        
        tti = driver.execute_script("""
            return new Promise((resolve) => {
                let lastLongTask = 0;
                
                // Monitor long tasks (>50ms)
                const observer = new PerformanceObserver((list) => {
                    for (const entry of list.getEntries()) {
                        lastLongTask = entry.startTime + entry.duration;
                    }
                });
                observer.observe({ entryTypes: ['longtask'] });
                
                // Check every 100ms if TTI reached
                const checkTTI = () => {
                    const now = performance.now();
                    const navStart = performance.timing.navigationStart;
                    const domInteractive = performance.timing.domInteractive - navStart;
                    
                    // TTI = max(domInteractive, lastLongTask + 5000ms quiet window)
                    if (now > Math.max(domInteractive, lastLongTask + 5000)) {
                        resolve(now);
                    } else {
                        setTimeout(checkTTI, 100);
                    }
                };
                
                setTimeout(checkTTI, 1000);
            });
        """)
        
        print(f"TTI: {tti}ms")
        assert tti < 3500, f"TTI too slow: {tti}ms"
```

---

## **18.6 Security Testing Basics**

### **Common Web Vulnerabilities**

```python
class TestBasicSecurity:
    """Fundamental security checks for web applications"""
    
    def test_xss_protection(self, driver):
        """Cross-Site Scripting (XSS) prevention"""
        driver.get("https://example.com/search")
        
        # Attempt XSS injection
        xss_payload = "<script>alert('xss')</script>"
        search_box = driver.find_element(By.ID, "search")
        search_box.send_keys(xss_payload)
        search_box.submit()
        
        # Check if script was executed (it shouldn't be)
        # 1. No alert should appear (would hang test)
        # 2. Output should be sanitized
        results = driver.find_element(By.ID, "results")
        result_text = results.text
        
        # Script tags should be escaped or removed
        assert "<script>" not in result_text or "&lt;script&gt;" in result_text
        
        # Or check that alert didn't fire by verifying page state
        assert driver.find_element(By.TAG_NAME, "body") is not None
    
    def test_csrf_protection(self, driver):
        """Cross-Site Request Forgery protection"""
        driver.get("https://example.com/profile")
        
        # Login first
        # ... login steps ...
        
        # Check for CSRF token in forms
        form = driver.find_element(By.ID, "update-profile")
        
        # Look for hidden CSRF token field
        try:
            csrf_token = form.find_element(By.NAME, "csrf_token")
            assert len(csrf_token.get_attribute("value")) > 10
        except NoSuchElementException:
            # Or check meta tag
            meta = driver.find_element(By.CSS_SELECTOR, "meta[name='csrf-token']")
            assert meta.get_attribute("content") is not None
    
    def test_sql_injection_protection(self, driver):
        """SQL injection prevention in search/login"""
        driver.get("https://example.com/login")
        
        # SQL injection attempt
        sql_payload = "' OR '1'='1"
        
        username = driver.find_element(By.ID, "username")
        password = driver.find_element(By.ID, "password")
        
        username.send_keys(sql_payload)
        password.send_keys("anything")
        password.submit()
        
        # Should NOT login successfully
        # Check for error message or still on login page
        assert "login" in driver.current_url or driver.find_element(By.CLASS_NAME, "error")
        
        # Should not show database error (information disclosure)
        page_source = driver.page_source.lower()
        assert "sql" not in page_source or "syntax" not in page_source
        assert "mysql" not in page_source
        assert "postgresql" not in page_source
    
    def test_secure_cookies(self, driver):
        """Verify secure cookie attributes"""
        driver.get("https://example.com/login")
        
        # Login to set cookies
        # ... login ...
        
        cookies = driver.get_cookies()
        
        for cookie in cookies:
            if "session" in cookie["name"].lower() or "auth" in cookie["name"].lower():
                # Secure flag (HTTPS only)
                assert cookie.get("secure") == True, f"{cookie['name']} missing Secure flag"
                
                # HttpOnly flag (no JavaScript access)
                assert cookie.get("httpOnly") == True, f"{cookie['name']} missing HttpOnly flag"
                
                # SameSite attribute
                same_site = cookie.get("sameSite")
                assert same_site in ["Strict", "Lax"], f"{cookie['name']} missing SameSite"
    
    def test_clickjacking_protection(self, driver):
        """X-Frame-Options or CSP frame-ancestors"""
        driver.get("https://example.com")
        
        # Check for clickjacking protection headers
        # Note: Selenium can't directly check HTTP headers
        # Use browser DevTools Protocol or API check
        
        # Alternative: Try to iframe the site (should fail if protected)
        driver.execute_script("""
            var iframe = document.createElement('iframe');
            iframe.src = 'https://example.com';
            document.body.appendChild(iframe);
        """)
        
        # If X-Frame-Options is set to DENY or SAMEORIGIN,
        # the iframe won't load the content properly
        # This is a basic check - proper check requires HTTP header inspection
    
    def test_https_enforcement(self, driver):
        """Verify HTTPS is enforced"""
        # Try to access HTTP version
        driver.get("http://example.com")  # Note: http not https
        
        # Should redirect to HTTPS
        assert driver.current_url.startswith("https://")
        
        # Or check for HSTS header (requires header inspection)
        # Strict-Transport-Security: max-age=31536000; includeSubDomains
```

---

## **Chapter Summary**

### **Key Takeaways:**

1. **Functional Testing:** Verify business logic and user workflows. Test complete user journeys (E2E), API integrations, and feature functionality. Use data-driven approaches for comprehensive coverage.

2. **UI/UX Testing:** Beyond functionality—verify visual appearance, layout consistency, responsive behavior, and user experience. Check colors, fonts, spacing, and interactive elements.

3. **Cross-Browser Testing:** Applications must work consistently across Chrome, Firefox, Safari, and Edge. Account for rendering differences, JavaScript engine variations, and CSS support.

4. **Responsive Design:** Test across viewport sizes (mobile, tablet, desktop). Verify breakpoints, touch interactions, image scaling, and layout adaptations.

5. **Performance:** Measure Core Web Vitals (LCP, FID, CLS), page load times, and resource loading. Set performance budgets and fail tests on regression.

6. **Security Basics:** Test for XSS, CSRF, SQL injection, insecure cookies, and HTTPS enforcement. Security testing prevents common vulnerabilities.

### **Testing Checklist:**

- [ ] All user workflows function correctly
- [ ] UI elements render correctly across browsers
- [ ] Responsive layout works on all screen sizes
- [ ] Performance meets budget (< 3s load time)
- [ ] Security headers present and cookies secure
- [ ] Accessibility basics (alt text, labels, contrast)
- [ ] Error handling graceful and informative
- [ ] Data persistence and state management correct

---

## **📖 Next Chapter: Chapter 19 - Handling Advanced Web Scenarios**

Now that you understand the comprehensive types of web testing, **Chapter 19** dives into **advanced scenarios and edge cases** that you'll encounter in real-world automation.

In **Chapter 19**, you'll master:

- **Dynamic Content Handling:** Testing pages with AJAX, infinite scroll, and lazy loading
- **AJAX and Asynchronous Calls:** Waiting for and verifying async operations
- **Frames and iFrames:** Deep dive into nested frames, cross-origin frames, and frame switching strategies
- **Pop-ups, Alerts, and Modals:** Handling browser dialogs, custom modals, and multiple windows
- **File Upload and Download:** Testing file operations including drag-drop upload and download verification
- **Captchas and Anti-Bot Measures:** Strategies for testing protected applications (API bypasses, test modes)
- **Shadow DOM Testing:** Deep penetration into Shadow DOM trees and slots
- **Progressive Web App (PWA) Testing:** Service workers, offline functionality, push notifications, and add-to-home-screen

**Why Chapter 19 is Critical:** Real applications aren't static HTML pages. They have dynamic content, complex architectures, and security measures. This chapter provides the advanced techniques needed to automate modern web applications successfully.

**Continue to Chapter 19 to master the complex scenarios that separate junior testers from senior automation engineers!**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='17. modern_browser_automation_tools.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='19. handling_advanced_web_scenarios.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
